In [2]:
from gensim.models import word2vec
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from numpy import array
import gensim
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
import gensim
import re
import spacy
from scipy.spatial.distance import cosine

In [3]:
import gensim
import re
import spacy

# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','"','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

In [4]:
Qu = pd.read_excel('./data/Question.xlsx')
df = pd.read_csv('./data/class.csv')

In [5]:
Qu['سوال']

0               اين قاعة
1             وين القاعه
2        اين اجد القاعه 
3       اين مكان القاعه 
4         اين هي القاعه 
5               مين يدرس
6             مين دكتور 
7            من هو دكتور
8                من يدرس
9              من محاضر 
10       متى تبدا محاضره
11           متى محاضره 
12              متى درس 
13       متى اختبار مادة
14            متى اختبار
15       متى موعد اختبار
16            اقدر انزل 
17       هل استطيع تنزيل
18          هل اقدر انزل
19         هل اقدر اسجل 
20    هل من الممكن تسجيل
Name: سوال, dtype: object

In [6]:
trylist = list(df['المقرر'].unique())

In [7]:
def split_digits_and_words(s):
    result = []
    X = 0
    for c in s:
        if c.isdigit():
            if X ==1 :
                X = 0 
                result.append(" ")
            result.append(int(c))
        else:
            result.append(c)
            X = 1 
            
    s = ''.join(str(x) for x in result)

    return s






In [8]:
for i in range(len(trylist)):
    
    trylist[i] = split_digits_and_words(trylist[i])

In [9]:
trylist[0]

'MATH 115'

In [10]:
df.head(3)

,فتره,جاهزة,المستفيد,المحاضر,القاعة,الى,من,الايام,المسجلين,اعلى حد,التسلسل,النشاط,ساعات المقرر,اسم المقرر,المقرر,الشعبة,index
0,12,نعم,44,اسلام خان محمد نضرول,304,11:10 ص,09:45 ص,1,37,50,1,نظري,3,حساب التكامل,MATH115,846,1
1,12,نعم,44,اسلام خان محمد نضرول,304,09:15 ص,08:00 ص,4,37,50,1,نظري,3,حساب التكامل,MATH115,846,2
2,12,نعم,44,اسلام خان محمد نضرول,304,09:15 ص,08:00 ص,2 1,43,50,2,نظري,3,حساب التكامل,MATH115,847,3


In [11]:
count = 0
quList = []
for i in Qu['سوال']:
    for j in trylist:
        a = i + " " + j
        quList.append(a)
        # print(a)
        count+=1

In [12]:
sen1 = quList[0].split() #taking only the first Question in the list 'اين قاعة'
sen1

['اين', 'قاعة', 'MATH', '115']

In [13]:
# mode2 = gensim.models.Word2Vec.load('./data/aravec/GoogleNews-vectors-negative300.bin')

In [19]:
li = list(df['المقرر'].unique())


In [145]:
from scipy.spatial.distance import cosine
import nltk

#Load AraVec

model = gensim.models.Word2Vec.load('./data/aravec/wikipedia_cbow_100')
#import Dataset that have synonyms Question about the same thing 
# A = pd.read_excel('./data/Question.xlsx')
#converting the Dataframe into list 
# docs = 
# this Example of Question came from the user it also should include the name of class but this is how it should looks after cleaning 
target = " متى اختبار مادة MATH115" 
#spliting the sentence into words
for word in li :
    if word in target:
        searchItem = word
        target = target.replace(word, "")

maxPer = 0
maxSen = []
for j in range(len(Qu['سوال'])):
    sen1 = Qu['سوال'][j].split() #taking only the first Question in the list 'اين قاعة'
    sen2 = target.split()
    wordVec1 = 0
    for i in range(len(sen1)):
        #cleaning the word
        try:
            word = clean_str(sen1[i])
        # adding the vectors of all words
            wordVec1 += model.wv[ word]
        except:
            continue
        
    wordVec2 = 0
    for i in range(len(sen2)):
        try:
            word = clean_str(sen2[i])
            wordVec2 += model.wv[ word]
        except:
            continue
    #finding the cosine similarity of the two sentences 
    similarity = 1 - cosine(wordVec1, wordVec2)
    if similarity > maxPer :
        maxPer = similarity
        maxSen = sen1
        seachLoc = j
    # print(similarity)

print( maxSen)
print( maxPer)

if Qu.loc[seachLoc][1] == 1 :
    # print(f"  وقت الاختبار في الفتره {list(df[df['المقرر'] == searchItem]['فتره'].unique())}")
    for i in range(len(df[df['المقرر'] == searchItem]['الشعبة'])):
        print(f" شعبه {df[df['المقرر'] == searchItem]['الشعبة'][i] } في قاعه رقم {df[df['المقرر'] == searchItem]['القاعة'][i]}")

if Qu.loc[seachLoc][1] == 2 :
    # print(f"  وقت الاختبار في الفتره {list(df[df['المقرر'] == searchItem]['فتره'].unique())}")
    print(f"يدرسها {list(df[df['المقرر'] == searchItem]['المحاضر'].unique())}")

if Qu.loc[seachLoc][1] == 3 :
    # print(f"  وقت الاختبار في الفتره {list(df[df['المقرر'] == searchItem]['فتره'].unique())}")
    for i in range(len(df[df['المقرر'] == searchItem]['الشعبة'])):
        print(f" شعبه {df[df['المقرر'] == searchItem]['الشعبة'][i] } تبدا من {df[df['المقرر'] == searchItem]['من'][i]} الى {df[df['المقرر'] == searchItem]['الى'][i]}")
if Qu.loc[seachLoc][1] == 4 :
    print(f"  وقت الاختبار في الفتره {list(df[df['المقرر'] == searchItem]['فتره'].unique())}")


['متى', 'اختبار', 'مادة']
1
  وقت الاختبار في الفتره ['12']


In [146]:
#متى اختبار مادة

if Qu.loc[seachLoc][1] == 4 :
    print(f"  وقت الاختبار في الفتره {list(df[df['المقرر'] == searchItem]['فتره'].unique())}")

  وقت الاختبار في الفتره ['12']


In [144]:
# متى محاضره 
if Qu.loc[seachLoc][1] == 3 :
    # print(f"  وقت الاختبار في الفتره {list(df[df['المقرر'] == searchItem]['فتره'].unique())}")
    for i in range(len(df[df['المقرر'] == searchItem]['الشعبة'])):
        print(f" شعبه {df[df['المقرر'] == searchItem]['الشعبة'][i] } تبدا من {df[df['المقرر'] == searchItem]['من'][i]} الى {df[df['المقرر'] == searchItem]['الى'][i]}")

 شعبه 846 تبدا من 09:45 ص الى 11:10 ص
 شعبه 846 تبدا من 08:00 ص الى 09:15 ص
 شعبه 847 تبدا من 08:00 ص الى 09:15 ص
 شعبه 848 تبدا من 12:00 م الى 01:15 م
 شعبه 848 تبدا من 11:30 ص الى 12:45 م
 شعبه 2278 تبدا من 09:45 ص الى 11:10 ص
 شعبه 2278 تبدا من 08:00 ص الى 09:15 ص
 شعبه 2279 تبدا من 08:00 ص الى 09:15 ص
 شعبه 2280 تبدا من 11:30 ص الى 12:45 م
 شعبه 2280 تبدا من 12:00 م الى 01:15 م


In [142]:
#من هو دكتور
if Qu.loc[seachLoc][1] == 2 :
    # print(f"  وقت الاختبار في الفتره {list(df[df['المقرر'] == searchItem]['فتره'].unique())}")
    print(f"يدرسها {list(df[df['المقرر'] == searchItem]['المحاضر'].unique())}")

يدرسها ['اسلام خان محمد نضرول', 'لم يحدد من الكلية']


In [140]:
#اين مكان القاعه 
if Qu.loc[seachLoc][1] == 1 :
    # print(f"  وقت الاختبار في الفتره {list(df[df['المقرر'] == searchItem]['فتره'].unique())}")
    for i in range(len(df[df['المقرر'] == searchItem]['الشعبة'])):
        print(f" شعبه {df[df['المقرر'] == searchItem]['الشعبة'][i] } في قاعه رقم {df[df['المقرر'] == searchItem]['القاعة'][i]}")

 شعبه 846 في قاعه رقم 304
 شعبه 846 في قاعه رقم 304
 شعبه 847 في قاعه رقم 304
 شعبه 848 في قاعه رقم 302
 شعبه 848 في قاعه رقم 302
 شعبه 2278 في قاعه رقم لم تحدد من الكلية
 شعبه 2278 في قاعه رقم لم تحدد من الكلية
 شعبه 2279 في قاعه رقم لم تحدد من الكلية
 شعبه 2280 في قاعه رقم لم تحدد من الكلية
 شعبه 2280 في قاعه رقم لم تحدد من الكلية
